<a href="https://colab.research.google.com/github/HSS107048212/AI_HW4_TSMC/blob/main/sample_code_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [ ]:
pip install numpy matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Bidirectional
import keras
import seaborn as sns
import keras.backend as K
#from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation
os.getcwd()

'/content'

## Data

In [ ]:
# read training data - It is the aircraft engine run-to-failure data.
train_df = pd.read_csv(r'./train.csv')
print('viwe the data shape', train_df.shape)
train_df.head()

viwe the data shape (20631, 27)


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,189
3,1,4,0.0007,0.0000,100,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,187


In [ ]:
# read test data - It is the aircraft engine operating data without failure events recorded.
test_df = pd.read_csv(r'./test.csv')
print('viwe the data shape', test_df.shape)
test_df.head()

viwe the data shape (12680, 26)


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,0.0023,0.0003,100,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100,38.86,23.3735
1,1,2,-0.0027,-0.0003,100,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100,39.02,23.3916
2,1,3,0.0003,0.0001,100,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100,39.08,23.4166
3,1,4,0.0042,0.0000,100,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100,39.00,23.3737
4,1,5,0.0014,0.0000,100,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100,38.99,23.4130


## LSTM

In [ ]:
sequence_length = 30

In [ ]:
# function to reshape features into (samples, time steps, features)
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # for one id I put all the rows in a single matrix
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    # Iterate over two lists in parallel.
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 111 191 -> from row 111 to 191
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]

# pick the feature columns
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting'+ str(i) for i in range(1,4)]
sequence_cols.extend(sensor_cols)

# generator for the sequences
# transform each id of the train dataset in a sequence
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols))
           for id in train_df['id'].unique())

# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
print(seq_array.shape)

# function to generate labels
def gen_labels(id_df, seq_length, label):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # For one id I put all the labels in a single matrix.
    # For example:
    # [[1]
    # [4]
    # [1]
    # [5]
    # [9]
    # ...
    # [200]]
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    # I have to remove the first seq_length labels
    # because for one id the first sequence of seq_length size have as target
    # the last label (the previus ones are discarded).
    # All the next id's sequences will have associated step by step one label as target.
    return data_matrix[seq_length:num_elements, :]

# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL'])
             for id in train_df['id'].unique()]

label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape

(17631, 30, 24)


(17631, 1)

In [ ]:
def r2_keras(y_true, y_pred):
    """Coefficient of Determination
    """
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()
model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae',r2_keras])
print(model.summary())

# fit the network
history = model.fit(seq_array, label_array, epochs=30, batch_size=200, validation_split=0.05, verbose=2,)

# list all data in history
print(history.history.keys())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 100)           50000     
                                                                 
 dropout (Dropout)           (None, 30, 100)           0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                30200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 80251 (313.48 KB)
Trainable params: 80251 

In [ ]:
# We pick the last sequence for each id in the test data
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:]
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)

y_pred_test = model.predict(seq_array_test_last)

test_set = pd.DataFrame(y_pred_test, columns=['RUL'])
test_set.index = test_set.index+1
test_set.to_csv('my_submission.csv', index_label='id')


4/4 [==============================] - 1s 5ms/step


In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Bidirectional
import keras
import seaborn as sns
import keras.backend as K
#from keras.layers.core import Activation
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation
os.getcwd()
# read training data - It is the aircraft engine run-to-failure data.
train_df = pd.read_csv(r'./train.csv')
print('viwe the data shape', train_df.shape)
train_df.head()
# read test data - It is the aircraft engine operating data without failure events recorded.
test_df = pd.read_csv(r'./test.csv')
print('viwe the data shape', test_df.shape)
test_df.head()
sequence_length = 30
# function to reshape features into (samples, time steps, features)
def gen_sequence(id_df, seq_length, seq_cols):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # for one id I put all the rows in a single matrix
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    # Iterate over two lists in parallel.
    # For example id1 have 192 rows and sequence_length is equal to 50
    # so zip iterate over two following list of numbers (0,112),(50,192)
    # 0 50 -> from row 0 to row 50
    # 1 51 -> from row 1 to row 51
    # 2 52 -> from row 2 to row 52
    # ...
    # 111 191 -> from row 111 to 191
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]

# pick the feature columns
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting'+ str(i) for i in range(1,4)]
sequence_cols.extend(sensor_cols)

# generator for the sequences
# transform each id of the train dataset in a sequence
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols))
           for id in train_df['id'].unique())

# generate sequences and convert to numpy array
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
print(seq_array.shape)

# function to generate labels
def gen_labels(id_df, seq_length, label):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    # For one id I put all the labels in a single matrix.
    # For example:
    # [[1]
    # [4]
    # [1]
    # [5]
    # [9]
    # ...
    # [200]]
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    # I have to remove the first seq_length labels
    # because for one id the first sequence of seq_length size have as target
    # the last label (the previus ones are discarded).
    # All the next id's sequences will have associated step by step one label as target.
    return data_matrix[seq_length:num_elements, :]

# generate labels
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL'])
             for id in train_df['id'].unique()]

label_array = np.concatenate(label_gen).astype(np.float32)
label_array.shape
def r2_keras(y_true, y_pred):
    """Coefficient of Determination
    """
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()
model.add(LSTM(
         input_shape=(sequence_length, nb_features),
         units=100,
         return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(
          units=50,
          return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae',r2_keras])
print(model.summary())

# fit the network
history = model.fit(seq_array, label_array, epochs=30, batch_size=200, validation_split=0.05, verbose=2,)

# list all data in history
print(history.history.keys())
# We pick the last sequence for each id in the test data
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:]
                       for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]

seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)

y_pred_test = model.predict(seq_array_test_last)

test_set = pd.DataFrame(y_pred_test, columns=['RUL'])
test_set.index = test_set.index+1
test_set.to_csv('my_submission.csv', index_label='id')


In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.layers import Activation
import keras.backend as K
import matplotlib.pyplot as plt

# 讀取訓練數據和測試數據
train_df = pd.read_csv(r'./train.csv')
test_df = pd.read_csv(r'./test.csv')

sequence_length = 30

# 特徵列選擇
sensor_cols = ['s' + str(i) for i in range(1,22)]
setting_cols = ['setting'+ str(i) for i in range(1,4)]
sequence_cols = setting_cols + sensor_cols

# 生成序列的函數
def gen_sequence(id_df, seq_length, seq_cols):
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]

# 生成標籤的函數
def gen_labels(id_df, seq_length, label):
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    return data_matrix[seq_length:num_elements, :]

# 生成訓練數據和標籤
seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) for id in train_df['id'].unique())
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)

# 定義R2 Keras回歸指標
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return (1 - SS_res/(SS_tot + K.epsilon()))

# 建立模型
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()
model.add(Bidirectional(LSTM(units=100, return_sequences=True), input_shape=(sequence_length, nb_features)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=50, return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(units=100))
model.add(Activation("relu"))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', r2_keras])

# 訓練模型
history = model.fit(seq_array, label_array, epochs=100, batch_size=100, validation_split=0.05, verbose=2)

# 預測測試集
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]
seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
y_pred_test = model.predict(seq_array_test_last)

# 儲存預測結果
test_set = pd.DataFrame(y_pred_test, columns=['RUL'])
test_set.index = test_set.index+1
test_set.to_csv('my_submission.csv', index_label='id')


Epoch 1/100
168/168 - 9s - loss: 5656.4126 - mae: 58.8917 - r2_keras: -4.9257e-01 - val_loss: 3711.9014 - val_mae: 49.9624 - val_r2_keras: -2.6007e+00 - 9s/epoch - 56ms/step
Epoch 2/100
168/168 - 2s - loss: 3841.3154 - mae: 50.5358 - r2_keras: -1.8239e-02 - val_loss: 3710.5085 - val_mae: 49.8828 - val_r2_keras: -2.5968e+00 - 2s/epoch - 14ms/step
Epoch 3/100
168/168 - 2s - loss: 3829.9761 - mae: 50.4459 - r2_keras: -1.5410e-02 - val_loss: 3711.1377 - val_mae: 49.9326 - val_r2_keras: -2.5991e+00 - 2s/epoch - 12ms/step
Epoch 4/100
168/168 - 2s - loss: 3835.3921 - mae: 50.4519 - r2_keras: -1.7268e-02 - val_loss: 3715.5613 - val_mae: 50.0509 - val_r2_keras: -2.6065e+00 - 2s/epoch - 10ms/step
Epoch 5/100
168/168 - 2s - loss: 3830.0483 - mae: 50.4439 - r2_keras: -1.7133e-02 - val_loss: 3712.6995 - val_mae: 49.7972 - val_r2_keras: -2.5953e+00 - 2s/epoch - 10ms/step
Epoch 6/100
168/168 - 2s - loss: 3839.7117 - mae: 50.5125 - r2_keras: -1.7375e-02 - val_loss: 3719.4778 - val_mae: 50.1203 - val_r

In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Conv1D, MaxPooling1D, Flatten
from keras.layers import Activation
import keras.backend as K
import matplotlib.pyplot as plt

# 讀取並預處理數據
train_df = pd.read_csv(r'./train.csv')
test_df = pd.read_csv(r'./test.csv')
sequence_length = 30

sensor_cols = ['s' + str(i) for i in range(1,22)]
setting_cols = ['setting' + str(i) for i in range(1,4)]
sequence_cols = setting_cols + sensor_cols

# 數據標準化
scaler = StandardScaler()
train_df[sequence_cols] = scaler.fit_transform(train_df[sequence_cols])
test_df[sequence_cols] = scaler.transform(test_df[sequence_cols])

# 生成序列和標籤
def gen_sequence(id_df, seq_length, seq_cols):
    data_matrix = id_df[seq_cols].values
    num_elements = data_matrix.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_matrix[start:stop, :]

def gen_labels(id_df, seq_length, label):
    data_matrix = id_df[label].values
    num_elements = data_matrix.shape[0]
    return data_matrix[seq_length:num_elements, :]

seq_gen = (list(gen_sequence(train_df[train_df['id']==id], sequence_length, sequence_cols)) for id in train_df['id'].unique())
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
label_gen = [gen_labels(train_df[train_df['id']==id], sequence_length, ['RUL']) for id in train_df['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)

# 模型建立
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, nb_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(units=100, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=50, return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', r2_keras])

# 訓練模型
history = model.fit(seq_array, label_array, epochs=110, batch_size=200, validation_split=0.1, verbose=2)

# 預測並保存結果
seq_array_test_last = [test_df[test_df['id']==id][sequence_cols].values[-sequence_length:] for id in test_df['id'].unique() if len(test_df[test_df['id']==id]) >= sequence_length]
seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)
y_pred_test = model.predict(seq_array_test_last)
test_set = pd.DataFrame(y_pred_test, columns=['RUL'])
test_set.index = test_set.index+1
test_set.to_csv('my_submission.csv', index_label='id')


Epoch 1/110
80/80 - 16s - loss: 7006.2744 - mae: 65.3536 - r2_keras: -9.2474e-01 - val_loss: 5208.1948 - val_mae: 57.7794 - val_r2_keras: -3.2626e-01 - 16s/epoch - 197ms/step
Epoch 2/110
80/80 - 1s - loss: 2536.2534 - mae: 37.9476 - r2_keras: 0.3014 - val_loss: 2254.9302 - val_mae: 35.2496 - val_r2_keras: 0.4300 - 864ms/epoch - 11ms/step
Epoch 3/110
80/80 - 1s - loss: 1320.7590 - mae: 25.6220 - r2_keras: 0.6394 - val_loss: 1825.1820 - val_mae: 30.6266 - val_r2_keras: 0.5649 - 799ms/epoch - 10ms/step
Epoch 4/110
80/80 - 1s - loss: 1019.8317 - mae: 21.6144 - r2_keras: 0.7212 - val_loss: 1712.1676 - val_mae: 29.6648 - val_r2_keras: 0.3981 - 765ms/epoch - 10ms/step
Epoch 5/110
80/80 - 1s - loss: 920.1761 - mae: 20.3677 - r2_keras: 0.7477 - val_loss: 1752.0417 - val_mae: 28.0359 - val_r2_keras: 0.5534 - 774ms/epoch - 10ms/step
Epoch 6/110
80/80 - 1s - loss: 781.9423 - mae: 19.0232 - r2_keras: 0.7852 - val_loss: 1598.9059 - val_mae: 27.4004 - val_r2_keras: 0.5275 - 773ms/epoch - 10ms/step
Ep